In [17]:
!pip3 install scikit-learn
!pip3 install pandas 

import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score

import numpy as np

Load Dataset

In [18]:
dataset = pd.read_csv('./dataset/eucalyptus.csv')

# Delete whitespace in column names
dataset.columns = dataset.columns.str.strip()

Preprocessing Dataset

In [19]:
lable_encode = LabelEncoder()

dataset = pd.get_dummies(dataset, dtype=int, columns = ['Abbrev', 'Rep', 'Locality', 'Latitude', 'Altitude', 'Rainfall', 'Frosts', 'Year', 'Sp', 'PMCno'])

dataset['Utility'] = lable_encode.fit_transform(dataset['Utility'])
dataset = dataset.sample(frac=1)

Dropping unnecessary attributes, scaling and dividing the set

In [20]:
dataset_attributes = dataset.drop('Utility', axis=1)
dataset_attributes = dataset_attributes.drop(columns= ['Map_Ref'], axis=1)
dataset_class = dataset['Utility']

imputer = SimpleImputer()
imputed_dataset_attributes = pd.DataFrame(imputer.fit_transform(dataset_attributes))
imputed_dataset_attributes.columns = dataset_attributes.columns

scaler = StandardScaler()
training_set = imputed_dataset_attributes.head(600)
testing_set = imputed_dataset_attributes.tail(68)
training_set = scaler.fit_transform(training_set)
testing_set = scaler.fit_transform(testing_set)
testing_validation = dataset_class.tail(68)
training_validation = dataset_class.head(600)

Dropping certain amount of features using random forest classifier

In [21]:
model = RandomForestClassifier(random_state=42)

# Fit the model to the data
model.fit(imputed_dataset_attributes, dataset_class)

# Get the most important features
most_important_attributes = pd.DataFrame(
                            model.feature_importances_,
                            index = imputed_dataset_attributes.columns,
                            columns=['importance']
                        ).sort_values('importance', ascending=False)

dropped_collums_01 = most_important_attributes[most_important_attributes['importance'] < 0.01].index.tolist()
dropped_collums_015 = most_important_attributes[most_important_attributes['importance'] < 0.015].index.tolist()
dropped_collums_02 = most_important_attributes[most_important_attributes['importance'] < 0.02].index.tolist()
dataset_attributes_dropped_01 = imputed_dataset_attributes.drop(axis=1, columns = dropped_collums_01)
training_set_dropped_01 = dataset_attributes_dropped_01.head(600)
testing_set_dropped_01 = dataset_attributes_dropped_01.tail(68)
training_set_dropped_01 = scaler.fit_transform(training_set_dropped_01)
testing_set_dropped_01 = scaler.fit_transform(testing_set_dropped_01)

dataset_attributes_dropped_015 = imputed_dataset_attributes.drop(axis=1, columns = dropped_collums_015)
training_set_dropped_015 = dataset_attributes_dropped_015.head(600)
testing_set_dropped_015 = dataset_attributes_dropped_015.tail(68)
training_set_dropped_015 = scaler.fit_transform(training_set_dropped_015)
testing_set_dropped_015 = scaler.fit_transform(testing_set_dropped_015)

dataset_attributes_dropped_02 = imputed_dataset_attributes.drop(axis=1, columns = dropped_collums_02)
training_set_dropped_02 = dataset_attributes_dropped_02.head(600)
testing_set_dropped_02 = dataset_attributes_dropped_02.tail(68)
training_set_dropped_02 = scaler.fit_transform(training_set_dropped_02)
testing_set_dropped_02 = scaler.fit_transform(testing_set_dropped_02)

KNN

In [22]:
def knnModel(training_set, training_validation, testing_set, testing_validation, scaling, neighbors = 0, weight = 'uniform'):
    k_values = list(range(1, 20))
    a_scores = []
    k = neighbors

    if k == 0:
        for n in k_values:
            knn_model = KNeighborsClassifier(n_neighbors=n, weights=weight)
            knn_model.fit(training_set, training_validation)
            Y_predicted = knn_model.predict(testing_set)
            
            score = accuracy_score(testing_validation, Y_predicted)
            a_scores.append(score)

        k = k_values[np.argmax(a_scores)]

    knn_model = KNeighborsClassifier(n_neighbors=k, weights=weight)
    start_holdout = time.process_time()
    knn_model.fit(training_set, training_validation)
    Y_predicted = knn_model.predict(testing_set)
    end_holdout = time.process_time()

    accuracy = accuracy_score(testing_validation, Y_predicted)
    f1 = f1_score(testing_validation, Y_predicted, average='weighted')
    report = classification_report(testing_validation, Y_predicted, output_dict=True)
    precision = report['macro avg']['precision']
    recall = report['macro avg']['recall']

    start_cross = time.process_time()
    scores = cross_val_score(
            knn_model, 
            np.concatenate((training_set, testing_set), axis=0), 
            np.concatenate((training_validation, testing_validation), axis=0), 
            cv=5, 
            scoring='accuracy'
        )
    avg_cross_accuracy = scores.mean()
    end_cross = time.process_time()

    # Return metrics
    return {
        "scaling": str(scaling),
        "k": k,
        "weights": weight,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "avg_cross": avg_cross_accuracy,
        "runtime_holdout": end_holdout - start_holdout,
        "runtime_cross": end_cross - start_cross
    }


Neural Network

In [23]:
def neuralNetworkModel(training_set, training_validation, testing_set, testing_validation, scaling):
    hidden_layers = [(5, 3), (20, 10, 5)]
    activations = ['relu', 'identity', 'logistic', 'tanh']
    solvers = ['adam', 'lbfgs', 'sgd']

    results = []
    for hidden_layer in hidden_layers:
        for activation in activations:
            for solver in solvers:
        
                neural_network_model = MLPClassifier(hidden_layer, max_iter=100, activation=activation, solver=solver, random_state=42)

                start_holdout = time.process_time()
                neural_network_model.fit(training_set, training_validation)
                Y_predicted = neural_network_model.predict(testing_set)
                end_holdout = time.process_time()
                accuracy = accuracy_score(testing_validation, Y_predicted)
                f1 = f1_score(testing_validation, Y_predicted, average='weighted')
                report = classification_report(testing_validation, Y_predicted, output_dict=True)
                precision = report['macro avg']['precision']
                recall = report['macro avg']['recall']

                start_cross = time.process_time()
                scores = cross_val_score(
                        neural_network_model, 
                        np.concatenate((training_set, testing_set), axis=0), 
                        np.concatenate((training_validation, testing_validation), axis=0), 
                        cv=5, 
                        scoring='accuracy'
                    )
                avg_cross_accuracy = scores.mean()
                end_cross = time.process_time()

                results.append({
                    "scaling": str(scaling),
                    "hidden_layers": hidden_layer,
                    "activation": activation,
                    "solver": solver,
                    "accuracy": accuracy,
                    "precision": precision,
                    "recall": recall,
                    "f1_score": f1,
                    "avg_cross": avg_cross_accuracy,
                    "runtime_holdout": end_holdout - start_holdout,
                    "runtime_cross": end_cross - start_cross
                })

    results_df = pd.DataFrame(results)
    markdown_table = results_df.to_markdown(index=False)
    print(markdown_table)

Random Forest


In [24]:
def randomForestModel(training_set, training_validation, testing_set, testing_validation, scaling = True):
    # Define the configurations
    leaf_nodes = [5, 100, 5000]
    max_depth = [5, 20, 1000]
    max_features = [5, 20]

    # Initialize a list to store the results
    results = []

    # Loop over all configurations
    for l in leaf_nodes:
        for d in max_depth:
            for f in max_features:
                # Initialize the MLP model
                # Initializing the Random Forest model
                random_forest_model = RandomForestClassifier(
                                        max_leaf_nodes=l,
                                        max_depth=d,
                                        max_features=f,
                                        random_state=42
                                    )

                # Train and evaluate the model
                start_holdout = time.process_time()
                random_forest_model.fit(training_set, training_validation)
                Y_predicted = random_forest_model.predict(testing_set)
                end_holdout = time.process_time()

                accuracy = accuracy_score(testing_validation, Y_predicted)
                f1 = f1_score(testing_validation, Y_predicted, average='weighted')
                report = classification_report(testing_validation, Y_predicted, output_dict=True)
                precision = report['macro avg']['precision']
                recall = report['macro avg']['recall']
                start_cross = time.process_time()
                scores = cross_val_score(
                    random_forest_model, 
                    np.concatenate((training_set, testing_set), axis=0), 
                    np.concatenate((training_validation, testing_validation), axis=0), 
                    cv=5, 
                    scoring='accuracy'
                    )
                end_cross = time.process_time()
                avg_cross_accuracy = scores.mean()
                # Append the results to the list
                results.append({
                    "scaling": str(scaling),
                    "leaf_nodes": l,
                    "max_depth": d,
                    "max_features": f,
                    "Accuracy": accuracy,
                    "Precision": precision,
                    "Recall": recall,
                    "F1": f1,
                    "avg_cross": avg_cross_accuracy,
                    "runtime_holdout": end_holdout - start_holdout,
                    "runtime_cross": end_cross - start_cross
                })

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)

    # Convert the DataFrame to a Markdown table
    markdown_table = results_df.to_markdown(index=False)

    # Print the Markdown table
    print(markdown_table)

KNN Tests

In [25]:
results = []
results.append(knnModel(training_set, training_validation, testing_set, testing_validation, True)) #optimal + scaling + all features
results.append(knnModel(training_set, training_validation, testing_set, testing_validation, True, weight='distance')) #optimal + scaling + all features
results.append(knnModel(training_set, training_validation, testing_set, testing_validation, True, neighbors=5)) #5 + scaling + all features
results.append(knnModel(training_set, training_validation, testing_set, testing_validation, True, neighbors=10)) #10 + scaling + all features
results.append(knnModel(imputed_dataset_attributes.head(600), training_validation, imputed_dataset_attributes.tail(68), testing_validation, False)) #optimal + all features
results.append(knnModel(imputed_dataset_attributes.head(600), training_validation, imputed_dataset_attributes.tail(68), testing_validation, False, weight='distance')) #optimal + all features
results.append(knnModel(imputed_dataset_attributes.head(600), training_validation, imputed_dataset_attributes.tail(68), testing_validation, False, neighbors=5)) #5 + all features
results.append(knnModel(imputed_dataset_attributes.head(600), training_validation, imputed_dataset_attributes.tail(68), testing_validation, False, neighbors=10)) #10 + all features
results.append(knnModel(training_set_dropped_01, training_validation, testing_set_dropped_01, testing_validation, True)) #optimal + scaling + 0.01
results.append(knnModel(training_set_dropped_01, training_validation, testing_set_dropped_01, testing_validation, True, weight='distance')) #optimal + scaling + 0.01
results.append(knnModel(training_set_dropped_01, training_validation, testing_set_dropped_01, testing_validation, True, neighbors=5)) #5 + scaling + 0.01
results.append(knnModel(training_set_dropped_01, training_validation, testing_set_dropped_01, testing_validation, True, neighbors=10)) #10 + scaling + 0.01
results.append(knnModel(training_set_dropped_015, training_validation, testing_set_dropped_015, testing_validation, True)) #optimal + scaling + 0.015
results.append(knnModel(training_set_dropped_015, training_validation, testing_set_dropped_015, testing_validation, True, weight='distance')) #optimal + scaling + 0.015
results.append(knnModel(training_set_dropped_015, training_validation, testing_set_dropped_015, testing_validation, True, neighbors=5)) #5 + scaling + 0.015
results.append(knnModel(training_set_dropped_015, training_validation, testing_set_dropped_015, testing_validation, True, neighbors=10)) #10 + scaling + 0.015
results.append(knnModel(training_set_dropped_02, training_validation, testing_set_dropped_02, testing_validation, True)) #optimal + scaling + 0.02
results.append(knnModel(training_set_dropped_02, training_validation, testing_set_dropped_02, testing_validation, True, weight='distance')) #optimal + scaling + 0.02
results.append(knnModel(training_set_dropped_02, training_validation, testing_set_dropped_02, testing_validation, True, neighbors=5)) #5 + scaling + 0.02
results.append(knnModel(training_set_dropped_02, training_validation, testing_set_dropped_02, testing_validation, True, neighbors=10)) #10 + scaling + 0.02

print(results)

[{'scaling': 'True', 'k': 11, 'weights': 'uniform', 'accuracy': 0.4411764705882353, 'precision': 0.48804855275443515, 'recall': 0.41873015873015873, 'f1_score': 0.43178809985532673, 'avg_cross': 0.42216361800022445, 'runtime_holdout': 0.0, 'runtime_cross': 0.09375}, {'scaling': 'True', 'k': 16, 'weights': 'distance', 'accuracy': 0.39705882352941174, 'precision': 0.5213511420828494, 'recall': 0.3790476190476191, 'f1_score': 0.3842341482047364, 'avg_cross': 0.46259679048367186, 'runtime_holdout': 0.0, 'runtime_cross': 0.125}, {'scaling': 'True', 'k': 5, 'weights': 'uniform', 'accuracy': 0.3382352941176471, 'precision': 0.3456043956043956, 'recall': 0.3057142857142857, 'f1_score': 0.33990552199004564, 'avg_cross': 0.39359218942879587, 'runtime_holdout': 0.0, 'runtime_cross': 0.125}, {'scaling': 'True', 'k': 10, 'weights': 'uniform', 'accuracy': 0.36764705882352944, 'precision': 0.4125, 'recall': 0.35428571428571426, 'f1_score': 0.3470217133282808, 'avg_cross': 0.4041521714734597, 'runtime

Neural Network Tests

In [28]:
neuralNetworkModel(training_set, training_validation, testing_set, testing_validation, True)
neuralNetworkModel(training_set_dropped_01, training_validation, testing_set_dropped_01, testing_validation, True)
neuralNetworkModel(training_set_dropped_015, training_validation, testing_set_dropped_015, testing_validation, True) #scaling + 0.015


C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in 

| scaling   | hidden_layers   | activation   | solver   |   accuracy |   precision |   recall |   f1_score |   avg_cross |   runtime_holdout |   runtime_cross |
|:----------|:----------------|:-------------|:---------|-----------:|------------:|---------:|-----------:|------------:|------------------:|----------------:|
| True      | (5, 3)          | relu         | adam     |   0.323529 |   0.206667  | 0.406667 |  0.209482  |    0.493985 |          0.109375 |        0.484375 |
| True      | (5, 3)          | relu         | lbfgs    |   0.544118 |   0.660519  | 0.575238 |  0.514124  |    0.592829 |          0.046875 |        0.21875  |
| True      | (5, 3)          | relu         | sgd      |   0.308824 |   0.395415  | 0.313333 |  0.240009  |    0.34123  |          0.09375  |        0.421875 |
| True      | (5, 3)          | identity     | adam     |   0.367647 |   0.367118  | 0.445397 |  0.315617  |    0.482067 |          0.09375  |        0.4375   |
| True      | (5, 3)          | id

C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Random Forest Tests

In [27]:
randomForestModel(training_set, training_validation, testing_set, testing_validation)
randomForestModel(dataset_attributes.head(600), training_validation, dataset_attributes.tail(68), testing_validation, scaling=False)
randomForestModel(training_set_dropped_01, training_validation, testing_set_dropped_01, testing_validation)
randomForestModel(training_set_dropped_015, training_validation, testing_set_dropped_015, testing_validation)
randomForestModel(training_set_dropped_02, training_validation, testing_set_dropped_02, testing_validation)

C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classific

| scaling   |   leaf_nodes |   max_depth |   max_features |   Accuracy |   Precision |   Recall |       F1 |   avg_cross |   runtime_holdout |   runtime_cross |
|:----------|-------------:|------------:|---------------:|-----------:|------------:|---------:|---------:|------------:|------------------:|----------------:|
| True      |            5 |           5 |              5 |   0.279412 |    0.446875 | 0.313333 | 0.170165 |    0.43857  |          0.109375 |        0.515625 |
| True      |            5 |           5 |             20 |   0.294118 |    0.144481 | 0.366667 | 0.145109 |    0.511996 |          0.109375 |        0.59375  |
| True      |            5 |          20 |              5 |   0.279412 |    0.446875 | 0.313333 | 0.170165 |    0.43857  |          0.109375 |        0.546875 |
| True      |            5 |          20 |             20 |   0.294118 |    0.144481 | 0.366667 | 0.145109 |    0.511996 |          0.109375 |        0.53125  |
| True      |            5 |      

C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classific

| scaling   |   leaf_nodes |   max_depth |   max_features |   Accuracy |   Precision |   Recall |       F1 |   avg_cross |   runtime_holdout |   runtime_cross |
|:----------|-------------:|------------:|---------------:|-----------:|------------:|---------:|---------:|------------:|------------------:|----------------:|
| True      |            5 |           5 |              5 |   0.308824 |    0.344481 | 0.38     | 0.172683 |    0.440085 |          0.09375  |        0.484375 |
| True      |            5 |           5 |             20 |   0.323529 |    0.246228 | 0.393333 | 0.221134 |    0.529941 |          0.125    |        0.5625   |
| True      |            5 |          20 |              5 |   0.308824 |    0.344481 | 0.38     | 0.172683 |    0.440085 |          0.109375 |        0.515625 |
| True      |            5 |          20 |             20 |   0.323529 |    0.246228 | 0.393333 | 0.221134 |    0.529941 |          0.109375 |        0.546875 |
| True      |            5 |      

C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classific

| scaling   |   leaf_nodes |   max_depth |   max_features |   Accuracy |   Precision |   Recall |       F1 |   avg_cross |   runtime_holdout |   runtime_cross |
|:----------|-------------:|------------:|---------------:|-----------:|------------:|---------:|---------:|------------:|------------------:|----------------:|
| True      |            5 |           5 |              5 |   0.367647 |    0.337424 | 0.431111 | 0.288175 |    0.556918 |          0.109375 |        0.578125 |
| True      |            5 |           5 |             20 |   0.455882 |    0.416465 | 0.482222 | 0.394911 |    0.540433 |          0.15625  |        0.71875  |
| True      |            5 |          20 |              5 |   0.367647 |    0.337424 | 0.431111 | 0.288175 |    0.556918 |          0.125    |        0.5625   |
| True      |            5 |          20 |             20 |   0.455882 |    0.416465 | 0.482222 | 0.394911 |    0.540433 |          0.15625  |        0.703125 |
| True      |            5 |      

C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classific

| scaling   |   leaf_nodes |   max_depth |   max_features |   Accuracy |   Precision |   Recall |       F1 |   avg_cross |   runtime_holdout |   runtime_cross |
|:----------|-------------:|------------:|---------------:|-----------:|------------:|---------:|---------:|------------:|------------------:|----------------:|
| True      |            5 |           5 |              5 |   0.367647 |    0.337424 | 0.431111 | 0.288175 |    0.556918 |          0.125    |        0.609375 |
| True      |            5 |           5 |             20 |   0.455882 |    0.416465 | 0.482222 | 0.394911 |    0.540433 |          0.140625 |        0.6875   |
| True      |            5 |          20 |              5 |   0.367647 |    0.337424 | 0.431111 | 0.288175 |    0.556918 |          0.109375 |        0.546875 |
| True      |            5 |          20 |             20 |   0.455882 |    0.416465 | 0.482222 | 0.394911 |    0.540433 |          0.140625 |        0.65625  |
| True      |            5 |      

C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Victor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classific

| scaling   |   leaf_nodes |   max_depth |   max_features |   Accuracy |   Precision |   Recall |       F1 |   avg_cross |   runtime_holdout |   runtime_cross |
|:----------|-------------:|------------:|---------------:|-----------:|------------:|---------:|---------:|------------:|------------------:|----------------:|
| True      |            5 |           5 |              5 |   0.441176 |    0.424132 | 0.468889 | 0.388172 |    0.550915 |          0.125    |        0.578125 |
| True      |            5 |           5 |             20 |   0.455882 |    0.416465 | 0.482222 | 0.394911 |    0.540433 |          0.140625 |        0.65625  |
| True      |            5 |          20 |              5 |   0.441176 |    0.424132 | 0.468889 | 0.388172 |    0.550915 |          0.125    |        0.578125 |
| True      |            5 |          20 |             20 |   0.455882 |    0.416465 | 0.482222 | 0.394911 |    0.540433 |          0.140625 |        0.65625  |
| True      |            5 |      